# Sniffing packets
- sniff() function allows us to capture filtered packets of interest
- equivalent to tcpdump and wireshark
- also has built-in wireshark function to parse big and complex packet captures

In [ ]:
from scapy.all import *
conf.verb=1
conf.color_theme = RastaTheme()

In [ ]:
help(sniff)

In [ ]:
# count argument value will sniff that many packets
pkts = sniff(count=2)

In [ ]:
pkts

In [ ]:
pkts.summary()

## Sniffing ping/icmp packets 
### Generate icmp packets using ping
1. open a terminal and run $ arping [ip of another vm/gateway]
2. sniff the packets using scapy

In [ ]:
# note: run ping google.com on a terminal to generate icmp packets
pkts = sniff(count=2, filter="icmp")

In [ ]:
pkts

In [ ]:
pkts.summary()

In [ ]:
pkts.show()

In [ ]:
pkts[0].show()

In [ ]:
hexdump(pkts[0])

## Sniffing arp packets 
### Generate arp packets using arping command
1. open a terminal and run $ arping [ip of another vm/gateway]
2. sniff the packets using scapy

In [ ]:
# ping a non-existant private ip or gateway or use arping
pkts = sniff(count=5, filter="arp")

In [ ]:
pkts.summary()

In [ ]:
pkts[0].show()

In [ ]:
pkts[1].show()

In [ ]:
pkts[0].command()